In [ ]:
import pandas as pd
from datetime import datetime 
import yfinance as yf

In [ ]:
# Set up dates
purchase_date = datetime(2023, 8, 21)
sell_date = datetime(2023, 11, 20)
month_one_end_date = datetime(2023, 9, 21)
month_two_end_date = datetime(2023, 10, 21)
month_three_end_date = datetime(2023, 11, 18)

In [ ]:
# Define methods
def get_stock_data(df_portfolio: pd.DataFrame) -> pd.DataFrame:
    """Get stock data from Yahoo Finance API for a list of tickers and a date range
    
    Args:
        df_portfolio: (pd.DataFrame): List of stock tickers
        
    Returns:
        pd.DataFrame: Stock data for tickers and date range
    """
    tickers = df_portfolio.index.to_list()
    stock_data = yf.download(tickers, start=purchase_date, end=sell_date, interval="1d", auto_adjust=True, prepost=False, threads=True, proxy=None, group_by='ticker')

    # Set up date index for df_stock_data from Purchase_date to sell_date
    stock_data = stock_data.reindex(pd.date_range(start=purchase_date, end=sell_date, freq='D'))
    stock_data = stock_data.ffill()
    stock_data = stock_data.dropna()
    
    for ticker in tickers:
        # add new column for initial investment to df_stock_data for each group
        stock_data[ticker, 'Initial Investment'] = df_portfolio.loc[ticker, 'AMOUNT_INVESTED']
        stock_data[ticker, 'Purchase Price'] = stock_data.loc[purchase_date, (ticker, 'Close')]
        stock_data[ticker, 'Shares'] = stock_data[ticker, 'Initial Investment'] / stock_data[ticker, 'Purchase Price']
        stock_data[ticker, 'Current Value'] = stock_data[ticker, 'Shares'] * stock_data[ticker, 'Close']        
        stock_data[ticker, 'Percent Change'] = (stock_data[ticker, 'Close'] - stock_data[ticker, 'Purchase Price']) / stock_data[ticker, 'Purchase Price']
        
    return stock_data

def extend_portfolio_data(df_portfolio: pd.DataFrame, df_stock_data: pd.DataFrame) -> None:
    """Extend portfolio data with stock data
    
    Args:
        df_portfolio (pd.DataFrame): Portfolio data
        df_stock_data (pd.DataFrame): Stock data
        
    Returns:
        None
    """
    tickers = df_portfolio.index.to_list()
    for ticker in tickers:
        if 'EXPECTED_RETURN_ARR' in df_portfolio.columns:
            df_portfolio.loc[ticker, 'ADJUSTED_EXPECTED_RETURN'] = df_portfolio.loc[ticker, 'EXPECTED_RETURN_ARR'] / 4
        else:
            df_portfolio.loc[ticker, 'EXPECTED_RETURN_ARR'] = 0
            df_portfolio.loc[ticker, 'ADJUSTED_EXPECTED_RETURN'] = 0
        
        df_portfolio.loc[ticker, 'PURCHASE_PRICE'] = df_stock_data.loc[purchase_date, (ticker, 'Close')]
        df_portfolio.loc[ticker, 'SHARES'] = df_stock_data.loc[purchase_date, (ticker, 'Shares')]
        df_portfolio.loc[ticker, 'MONTH_1_PRICE'] = df_stock_data.loc[month_one_end_date, (ticker, 'Close')]
        df_portfolio.loc[ticker, 'MONTH_1_VALUE'] = df_stock_data.loc[month_one_end_date, (ticker, 'Current Value')]
        df_portfolio.loc[ticker, 'MONTH_1_RETURN'] = df_stock_data.loc[month_one_end_date, (ticker, 'Current Value')] - df_stock_data.loc[purchase_date, (ticker, 'Current Value')]
        df_portfolio.loc[ticker, 'MONTH_2_PRICE'] = df_stock_data.loc[month_two_end_date, (ticker, 'Close')]
        df_portfolio.loc[ticker, 'MONTH_2_VALUE'] = df_stock_data.loc[month_two_end_date, (ticker, 'Current Value')]
        df_portfolio.loc[ticker, 'MONTH_2_RETURN'] = df_stock_data.loc[month_two_end_date, (ticker, 'Current Value')] - df_stock_data.loc[month_one_end_date, (ticker, 'Current Value')]
        df_portfolio.loc[ticker, 'MONTH_3_PRICE'] = df_stock_data.loc[month_three_end_date, (ticker, 'Close')]
        df_portfolio.loc[ticker, 'MONTH_3_VALUE'] = df_stock_data.loc[month_three_end_date, (ticker, 'Current Value')]
        df_portfolio.loc[ticker, 'MONTH_3_RETURN'] = df_stock_data.loc[month_three_end_date, (ticker, 'Current Value')] - df_stock_data.loc[month_two_end_date, (ticker, 'Current Value')]
        df_portfolio.loc[ticker, 'ACTUAL_RETURN'] = df_stock_data.loc[month_three_end_date, (ticker, 'Current Value')] - df_stock_data.loc[purchase_date, (ticker, 'Current Value')]
        df_portfolio.loc[ticker, 'ACTUAL_RETURN_PCT'] = df_stock_data.loc[month_three_end_date, (ticker, 'Percent Change')]
    

In [ ]:
df_original_portfolio = pd.read_csv('../data/portfolio/original_portfolio.csv', index_col=0)
original_stock_prices = get_stock_data(df_original_portfolio)
extend_portfolio_data(df_original_portfolio, original_stock_prices)

df_original_portfolio

In [ ]:
df_sentiment_portfolio = pd.read_csv('../data/portfolio/sentiment_portfolio.csv', index_col=0)
sentiment_stock_prices = get_stock_data(df_sentiment_portfolio)
extend_portfolio_data(df_sentiment_portfolio, sentiment_stock_prices)

df_sentiment_portfolio

In [ ]:
df_original_portfolio_returns = df_original_portfolio[['NAME', 'ADJUSTED_EXPECTED_RETURN', 'ACTUAL_RETURN_PCT']]
df_original_portfolio_returns.plot.bar(x='NAME', y=['ADJUSTED_EXPECTED_RETURN', 'ACTUAL_RETURN_PCT'], rot=0, figsize=(20,10))